In [2]:
import numpy as np
total = [0]
def my_matmult(m1, m2):
    ans_mat = np.empty((0,m2.shape[1]))
    for row in m1:
        ans_row = np.array([])
        for col in m2.T:
            ans_row = np.append(ans_row, np.dot(row, col))
            total[0] += len(row)*2
        ans_mat = np.vstack((ans_mat,ans_row))
    return ans_mat

a = np.random.rand(3, 2)
b = np.random.rand(2, 3)

print(np.array_equal(a@b, my_matmult(a,b)))


True


In [3]:
def tensor_product(m1, m2):
  ans = np.empty( (0, m1.shape[-2], m2.shape[-1]) )
  for i in range(m1.shape[0]):
    matmult = np.expand_dims(my_matmult(m1[i], m2[i]), axis=0)
    ans = np.vstack((ans, matmult))
  return ans

a = np.random.rand(2, 3, 4)
b = np.random.rand(2, 4, 2)

print(np.allclose(tensor_product(a,b), a @ b))


True


In [4]:
def nd_tensor_product(m1, m2):
  ans_shape = m1.shape[:-2] + (m1.shape[-2], m2.shape[-1])
  ans = np.empty(ans_shape)
  for i in range(m1.shape[0]):
    if len(m1.shape) == 3:
        ans[i] = my_matmult(m1[i], m2[i])

    else:
        ans[i] = nd_tensor_product(m1[i], m2[i])
  return ans

a = np.random.rand(12, 4, 64)
b = np.random.rand(12, 64, 4)

total = [0]

print(np.allclose(nd_tensor_product(a,b), a @ b))
print(total)

True
[24576]


In [3]:
import numpy as np

In [23]:
def absmax_quantization(m1, type=np.int8):
    matrix = np.empty((0, m1.shape[-1]))
    row_constants = []
    data_max = np.iinfo(type).max
    for row in m1:
        norm_range = round((data_max/2)**(1/2))

        row_constant = norm_range/np.max(abs(row))
        tow_type = (row * row_constant).astype(type)
        matrix.vstack(tow_type)
        row_constants += row_constant
    return matrix, row_constants

# def zeropoint_quantization(m1, type=np.int8):
#     return (m1 * 2* np.iinfo(type).max)/ (max(m1) - min(m1))


In [22]:


def my_matmul(m1, m2):
    m1, m1_const = absmax_quantization(m1, np.int16)
    m2, m2_const = absmax_quantization(m2, np.int16)

    ans = np.matmul(m1,m2)

    ans = (ans.astype(np.float64))/ (m1_const * m2_const)
    return ans

def nd_tensor_product(m1, m2):
    ans_shape = m1.shape[:-2] + (m1.shape[-2], m2.shape[-1])
    ans = np.empty(ans_shape)
    if len(ans_shape) < 3:
        return my_matmul(m1, m2)
    else:
        for i in range(ans_shape[0]):
            ans[i] = nd_tensor_product(m1[i], m2[i])
    return ans

a = np.random.uniform(-100, 100, size=(2, 3))
b = np.random.uniform(-100, 100, size=(3, 5))

print(a)
print(b)
print()
print(np.matmul(a,b))
nd_tensor_product(a,b)


[[ 83.59729455 -61.05295378  94.18623868]
 [ 85.78888278 -78.75996448  65.36486141]]
[[-45.01908378 -78.2509068   54.87254097   8.33575975 -37.11951929]
 [-41.50846467  85.60612985  -4.59093398  -7.84990055 -22.20440179]
 [ 28.97236314  26.89287862 -84.85988416  66.30060033  62.96910354]]

[[  1499.53867814  -9235.13210888  -3125.14725318   7420.71074612
    4183.37594457]
 [  1300.84280399 -11697.54433308   -477.81878473   5667.10295654
    2680.34253364]]
(2, 3)
(3, 5)


array([[  1484.73069851,  -9151.98435538,  -3134.81423583,
          7347.37465322,   4194.35192025],
       [  1302.15360565, -11686.90242897,   -459.64152218,
          5551.62313884,   2668.77513358]])